In [0]:
from  pyspark.sql.functions import *
from pyspark.sql.window import *
from delta.tables import DeltaTable

**Define paths**

In [0]:
silver_stores = "abfss://silver@stretailenvdev.dfs.core.windows.net/s_stores"
gold_stores = "abfss://gold@stretailenvdev.dfs.core.windows.net/reference_data/dim_store"


In [0]:
spark.read.format("delta").load(silver_stores).printSchema()


**CREATE DELTA LAKE TABLE IF NOT EXISTS**

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS retail_cata.gold.dim_store(
     store_key BIGINT GENERATED ALWAYS AS IDENTITY(START WITH 1 INCREMENT BY 1),
     StoreID  INT,
     StoreName STRING,
     Location STRING,
     OpeningDate TIMESTAMP ,
     StartDate TIMESTAMP,
     EndDate TIMESTAMP,
     IsActive BOOLEAN
)USING DELTA 
LOCATION "{gold_stores}"
""")

 **Load silver store data**

In [0]:
df_stores = spark.read.format("delta").load(silver_stores)


**Add SCD2 colums**

In [0]:
stg_store = df_stores.withColumn("StartDate", current_date())\
                     .withColumn("EndDate", lit(None).cast("date"))\
                     .withColumn("IsActive", lit(True))


In [0]:
stg_store.limit(5).display()

**Load Gold data**

In [0]:
if DeltaTable.isDeltaTable(spark, gold_stores):
    dim_stores = DeltaTable.forPath(spark, gold_stores)
else:
    stg_store.write.format("delta").mode("append").save(gold_stores)
    dim_stores = DeltaTable.forPath(spark, gold_stores)

    

**Merge SCD2**

In [0]:
(
    dim_stores.alias("t")
    .merge(
        stg_store.alias("s"),
        "t.StoreID = s.StoreID AND t.IsActive = TRUE"
    )
    .whenMatchedUpdate(
        condition ="""
            t.StoreID <> s.StoreID OR
            t.StoreName <> s.StoreName OR
            t.Location <> s.Location OR
            t.OpeningDate <> s.OpeningDate
        """,
        set={
            "EndDate" : "current_date()",
            "IsActive" : "False"
        }
    )
    .whenNotMatchedInsert(
        values={
            "StoreID" : "s.StoreID",
            "StoreName" : "s.StoreName",
            "Location" : "s.Location",
            "OpeningDate" : "s.OpeningDate",
            "StartDate" : "s.StartDate",
            "EndDate" : "s.EndDate",
            "IsActive" : "s.IsActive"
        }
    )
    .execute()
)

In [0]:
%sql
select * from retail_cata.gold.dim_store
limit 5;

**Define reference data paths**

In [0]:
silver_channel = "abfss://silver@stretailenvdev.dfs.core.windows.net/reference_data/channels"
silver_pytmethods = "abfss://silver@stretailenvdev.dfs.core.windows.net/reference_data/paymentmethods"
silver_prdtsubcate = "abfss://silver@stretailenvdev.dfs.core.windows.net/reference_data/productsubcategories"

In [0]:
gold_channel = "abfss://gold@stretailenvdev.dfs.core.windows.net/reference_data/dim_channel"
gold_pytmethods = "abfss://gold@stretailenvdev.dfs.core.windows.net/reference_data/dim_paymentmethod"
gold_prdtsubcate = "abfss://gold@stretailenvdev.dfs.core.windows.net/reference_data/dim_productsubcat"

**Define Delta lake Table for reference data**

DROP TABLE IF EXISTS retail_cata.gold.dim_channel2;
DROP TABLE IF EXISTS retail_cata.gold.dim_channel;
DROP TABLE IF EXISTS retail_cata.gold.dim_paymentmethod;
DROP TABLE IF EXISTS retail_cata.gold.dim_productsubcat;

**Dim_channel Table schema**

In [0]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS retail_cata.gold.dim_channel(
              channel_key INT,
              ChannelID INT,
              ChannelName STRING,
              Description STRING,
              LoadDate TIMESTAMP
          )USING DELTA 
          LOCATION "{gold_channel}"
          """)

**Dim_paymentmethod schema**

In [0]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS retail_cata.gold.dim_paymentmethod(
              paymentmethod_key INT,
              PaymentMethodID INT,
              MethodName STRING,
              LoadDate TIMESTAMP
          ) USING DELTA 
          LOCATION "{gold_pytmethods}"
          """)

**Dim_productsubcategory schema**

In [0]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS retail_cata.gold.dim_productsubcat(
              subcategory_key INT,
              SubCategoryID INT,
              CategoryID INT,
              SubCategoryName STRING,
              LoadDate TIMESTAMP
          )USING DELTA
          LOCATION "{gold_prdtsubcate}"
          """)

**load silver layer data**

In [0]:
df_channel = spark.read.format("delta").load(silver_channel)
df_paymentmethod = spark.read.format("delta").load(silver_pytmethods)
df_productsubcat = spark.read.format("delta").load(silver_prdtsubcate)

**Ingest surrogate key column data and loaddate data**

In [0]:
from pyspark.sql.window import *

stg_channel = (df_channel.withColumn("channel_key", 
                                     row_number().over(Window.orderBy('ChannelID')))
                                     .withColumn("LoadDate", current_timestamp()))

stg_paymentmethod = (df_paymentmethod.withColumn("paymentmethod_key", 
                                                 row_number().over(Window.orderBy   ('PaymentMethodID')))
                                                 .withColumn("LoadDate", current_timestamp()))

stg_productsubcat = (df_productsubcat.withColumn("subcategory_key", 
                                                 row_number().over(Window.orderBy('SubCategoryID')))
                                                 .withColumn("LoadDate", current_timestamp()))


In [0]:
stg_channel.display()
stg_paymentmethod.display()
stg_productsubcat.display()

In [0]:
stg_channel.write.format("delta").mode("overwrite").save(gold_channel)
stg_paymentmethod.write.format("delta").mode("overwrite").save(gold_pytmethods)
stg_productsubcat.write.format("delta").mode("overwrite").save(gold_prdtsubcate)

In [0]:
%sql
select * from retail_cata.gold.dim_channel

In [0]:
%sql
select * from retail_cata.gold.dim_productsubcat

In [0]:
%sql
select * from retail_cata.gold.dim_paymentmethod